# COGS 118A - Project Checkpoint

# Names

Hopefully your team is at least this good. Obviously you should replace these with your names.

- Yuqi Zhang
- Yiling Cao
- Xiaolong Zhou

# Abstract 
This section should be short and clearly stated. It should be a single paragraph <200 words.  It should summarize: 
- In addressing the need for automated music genre identification for enhanced user experience on music streaming platforms, we propose a machine learning approach to tackle genre classification and personalized music recommendation.
- Leveraging data features like tempo, rhythm, pitch, and lyrical content, our goal is to predict genre labels, a complex multiclass classification task that emulates the human ability to discern music genres. Concurrently, we aim to rank songs based on user preferences, transforming personalized music recommendation into an information retrieval task. Data used in this project will represent a variety of music genres and user listening habits, measured through both objective attributes of music and subjective user interactions.
- Machine learning algorithms will be utilized to make sense of this data, mapping musical features to genre labels and user preferences to song rankings.
- The performance of our genre prediction model will be gauged using standard classification metrics, such as accuracy, precision, recall, and F1-score, ensuring a quantitative measure of our success in mimicking the sensitive human ability to identify sound.

# Background
Music genre classification is a significant area of interest in digital music processing. The process involves extracting acoustic features of music, such as genre, rhythmic structure, harmonic content, duration, and the instruments played, using digital signal processing techniques. The classification plays a critical role in recommendation systems, audio libraries organization, and trend discovery<a name="ieee"></a>[<sup>[1]</sup>](#ieee)​.

Musical genres are human-made categories, defining different types of music based on shared features, including instrumentation, rhythmic structure, and harmonic content. These genre hierarchies help in managing the enormous volume of music available on the internet<a name="ssrn"></a>[<sup>[2]</sup>](#ssrn)​.

Various approaches, including Multiclass support vector machine, K-Nearest Neighbors, K-means clustering algorithm, and Convolutional neural network, are utilized in building music genre classification systems. The use of deep learning models, particularly convolutional neural networks, has gained popularity in automatic music genre classification, with a comparative study showing promising results against traditional classifiers<a name="raghav"></a>[<sup>[3]</sup>](#raghav).

Music recommendation systems are another crucial aspect of digital music platforms. These systems predict and push user preferences based on user behavior and music characteristics. The development of these algorithms has shifted from merely focusing on individual user preferences to incorporating mutual recommendations among users<a name="hindawi"></a>[<sup>[4]</sup>](#hindawi).

In summary, music genre identification and recommendation systems serve as essential components of digital music platforms, enhancing user experience by organizing music content and tailoring music selections to individual preferences. These systems leverage machine learning and deep learning methods to analyze music characteristics and user behavior, thereby enabling accurate genre classification and personalized music recommendations.

# Problem Statement

Human sound identify abbility is usually very sensitive. People usually identify a music genre "based on both objective and subjective measures"<a name="Lippens"></a>[<sup>[5]</sup>](#Lippens). When human subjectives is not possible to duplicate in machines, many music platforms still want machines to be able to identify such music genre as a source of feature preference. Music streaming platforms face two key challenges: accurate genre classification and personalized music recommendation. We propose to address these problems using machine learning. The genre prediction problem is a multiclass classification task, mapping input features (tempo, rhythm, pitch, lyrical content) to predefined genre labels. The recommendation problem can be defined as ranking songs based on user preferences, expressed as an information retrieval task. The problems can be mathematically defined and solved using ML algorithms. The genre prediction’s performance can be assessed using accuracy, precision, recall, and F1-score.

# Data
- Link: https://www.kaggle.com/datasets/vicsuperman/prediction-of-music-genre
- Number of observations: 50005 Number of variables: 15
- An observation consist of:
    - instance_id: A unique identifier for each instance.
    - popularity: The popularity of the track.
    - acousticness: A measure of the acoustic quality of the track.
    - danceability: A measure of the danceability of the track.
    - duration_ms: The duration of the track in milliseconds.
    - energy: A measure of the energy of the track.
    - instrumentalness: A measure of the instrumentalness of the track.
    - key: The key of the track.
    - liveness: A measure of the liveness of the track.
    - loudness: The loudness of the track.
    - mode: The mode of the track (Major or Minor).
    - speechiness: A measure of the speechiness of the track.
    - tempo: The tempo of the track.
    - valence: A measure of the musical positiveness conveyed by a track.
    - music_genre: The genre of the music.
- Critical variables: 
    - music_genre: This is likely the target variable because we are building a model to predict the genre of a track based on its other characteristics.

    - acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, valence: These are all numerical features that describe different aspects of the track, which could be important predictors in a model.

    - key, mode: These are categorical features that could also be important, depending on the specific genre we are trying to predict.
- Ways to represent:
    - Numerical variables (popularity, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, speechiness, tempo, valence) are represented as floating-point numbers.

    - Categorical variables (key, mode, music_genre) are represented as strings.

    - The instance_id is a unique identifier for each observation, represented as a floating-point number. 


# Set Up

In [7]:
# import pandas & numpy library
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report


# Data Cleaning

In [8]:
path = "music_genre.csv"
music_data = pd.read_csv(path)

# Drop unrelated columns
music_data = music_data.drop(['artist_name', 'track_name', 'obtained_date'], axis=1)

# Replace missing values
music_data.replace(['?', -1], np.nan, inplace=True)

# Convert data types
music_data['duration_ms'] = music_data['duration_ms'].astype(float)
music_data['tempo'] = music_data['tempo'].astype(float)

# Standardize text data
music_data['key'] = music_data['key'].str.lower().str.strip()
music_data['mode'] = music_data['mode'].str.lower().str.strip()
music_data['music_genre'] = music_data['music_genre'].str.lower().str.strip()

# Handle NaN values (this is just one approach, you'll need to decide what's best for your use case)
music_data.dropna(inplace=True)

print(music_data.head())

   instance_id  popularity  acousticness  danceability  duration_ms  energy  \
1      46652.0        31.0       0.01270         0.622     218293.0   0.890   
2      30097.0        28.0       0.00306         0.620     215613.0   0.755   
3      62177.0        34.0       0.02540         0.774     166875.0   0.700   
4      24907.0        32.0       0.00465         0.638     222369.0   0.587   
6      43760.0        46.0       0.02890         0.572     214408.0   0.803   

   instrumentalness key  liveness  loudness   mode  speechiness    tempo  \
1          0.950000   d     0.124    -7.043  minor       0.0300  115.002   
2          0.011800  g#     0.534    -4.617  major       0.0345  127.994   
3          0.002530  c#     0.157    -4.498  major       0.2390  128.014   
4          0.909000  f#     0.157    -6.266  major       0.0413  145.036   
6          0.000008   b     0.106    -4.294  major       0.3510  149.995   

   valence music_genre  
1    0.531  electronic  
2    0.333  electr

# Proposed Solution

By generating multiple music genre classification results using models such as neural network, KNN, and decision trees, we will maintain an accuracy percentage of our model to show how efficient such model can be with only subjective features we gained from the data. We have also tried implement sentimental data values from the dataset to try relate as close as possible to the true genre. These sentimental features are included as: popularity, acousticness, energy, intrumental, liveness, loudness, speechiness, tempo, valence. We might exclude "artist_name" and "track_name" since we want to focus on the element of a music genre as music it self instead of its title and author. For features such as key and mode, we will be doing one-hot-encoding to transform these categorical data to the same type as other features. As human, we are able to identify if such music is lively or not and base off that detect its genre. For this model, we want to simulate the same detection that our human can do toward the musics itself. We will be spliting our 50006 observations into train, valid, and test datasets. Using methods such as nested cross-validation would gave us accurate results from multiple different algorithm based models. We will be mainly observe a KNN and decision tree model by comparing and contrast each runtime and accuracy. We migh also need to do one-hot encoding since our labels are multiclass categorical. By tuning the parameters, we will find the best number of neighbors for KNN and best fitted tree depth and criterion for decision tree model. Idealy, a neural net model should be more accurate and widely used. However, with limited time and knowledge on neural networks, our team plan on test such algorithm model after we have a finite solution first. Our solution model can help our main project problem by providing a source to music platforms to quickly identify a genre when music data are being imported to the platform. It will also be helpful when the music platform tries to build a recommendation system and apply such model to help label what type of music is each user listening to. 

There is a benchmark model called "music_genre_classification" using Convolutional neural network(CNN) to perform music genre classification. [Link](https://github.com/ds7711/music_genre_classification/tree/master). This model has achieved an accuracy of 70%, which is comparable to the human accuracy. Such model can possiblly substitute human ears and identify music genres within 30 seconds of the music play. It will be more time saving and allow humans to apply their abilities on other fields with more time.

# Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

A mean accuracy from the nested cross validation would be the most desirable evaluation metrics that we will be using in this project. When we are doing a classification problem, we want to know how good is our model doing. In the benchmark model, the author provided their accuracy percentage on the test dataset. Our model first will classify test dataset music genres with the features only. Then we will take those labels accompany with the obervation ids to find whether or not the true label is the same as predicted label. Last, we will count up all the correct number of predicted labels and calculate the accuracy. The accuracy can be calculated using below formula: <br>
$
\begin{align}
x &= (x_1, ..., x_m), x_i \in \mathcal{R}^n, x \in \mathcal{R}^m \\
y & \in \mathcal{R}^{10}: \text{['Electronic', 'Anime', 'Jazz', 'Alternative', 'Country', 'Rap', 'Blues', 'Rock', 'Classical', 'Hip-Hop']} \\
accuracy &= 1 - error \\
error &= \frac{1}{n}\sum^n_{i=1}1(y_i \neq f(x_i))\\
\end{align}$

Since we planning on using nested cross-validation, each iteration will contain a best accuracy. In order to compare algorithms to algorithms, for example KNN to decision tree, we have to average such accuracy in total iterations and compare them. So, we might need to do multiple cross-validations to ensure our model is working efficiently. 

# Preliminary results

NEW SECTION!

Please show any preliminary results you have managed to obtain.

Examples would include:
- Analyzing the suitability of a dataset or alogrithm for prediction/solving your problem 
- Performing feature selection or hand-designing features from the raw data. Describe the features available/created and/or show the code for selection/creation
- Showing the performance of a base model/hyper-parameter setting.  Solve the task with one "default" algorithm and characterize the performance level of that base model.
- Learning curves or validation curves for a particular model
- Tables/graphs showing the performance of different models/hyper-parameters



# Ethics & Privacy

Data Permission & Privacy:
 The dataset: “Prediction of Music Genre” that we select is chosen from a public database website: Kaggle. Unlike other datasets that focus on human study, our chosen data collects only the description (artist name and track name, nothing on personal identities or ethnic background) and quantified structural information (popularity, duration, energy, key, etc.) of a variety of songs that are already posted online (collected through Spotify API); With the fact that it does not actually contain any of the song file, we eliminate all issues with regard to copyright and user privacy. Additionally, this particular dataset follows the CC0 1.0 Universal Public Domain Dedication, thus we have the right to use, modify and perform work on it. \

Potential Bias: 
 It is essential to notice that music genres are affected by various of aspects. Due to the limitation of variables in this dataset, we might not capture all the differences between genres to fully distinguish one from another and discover a feature bias. \
 
Implication:
 Since this model will only be used in this particular in-class project, we will interpret the result of our model in caution as for now, and we may keep researching and modifying the variables we use.


# Team Expectations 

* We expect each member to contribute equally during group discussion.
* We expect to conduct a thoughtful and sophisticated analysis to our data that leads to the best solution.
* We should respect each other’s idea.
* We expect to be unanimous on what we try to accomplish.
* We will thoroughly fulfill the timeline proposal.

# Project Timeline Proposal

UPDATE THE PROPOSAL TIMELINE ACCORDING TO WHAT HAS ACTUALLY HAPPENED AND HOW IT HAS EFFECTED YOUR FUTURE PLANS

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 5.10  |  8 PM |  Read project instructions; brainstorm for topics  | Draft a timeline that works for everyone; decide topic | 
| 5.14  |  10 AM |  Do background research for topic | Discuss and pick a dataset related to music genre; draft project proposal | 
| 5.17  | 10 AM  | Finalize and submit project proposal | Assign group members to each specific part; discuss approach to solution  |
| 5.27  | 10 AM  | Data wrangling, analysis; Model training | Be ready to submit project checkpoint   |
| 6.9   | 8 PM  | Model Testing, selecting; Complete Analysis | Draft results; draft final project file |
| 6.14  | Before 11:59 PM  | Finish final project| Submit final project |

# Footnotes
<a name="ieee"></a>1.[^](#ieee):Music Genre Classification and Recommendation by Using ... - IEEE Xplore, ieeexplore.ieee.org/document/8554016. Accessed 17 May 2023. https://ieeexplore.ieee.org/document/8554016<br> 
<a name="ssrn"></a>2.[^](#ssrn): Dias, Jessica, et al. “Music Genre Classification &amp; Recommendation System Using CNN.” SSRN, 18 May 2022, papers.ssrn.com/sol3/papers.cfm?abstract_id=4111849.https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4111849 <br>
<a name="raghav"></a>3.[^](#raghav): Agrawal, Raghav. “Music Genre Classification Project Using Machine Learning Techniques.” Analytics Vidhya, 6 Apr. 2022, https://www.analyticsvidhya.com/blog/2022/03/music-genre-classification-project-using-machine-learning-techniques/.  <br>
<a name="hindawi"></a>4.[^](#hindawi): Zhang, Yezi. “Music Recommendation System and Recommendation Model Based on Convolutional Neural Network.” Mobile Information Systems, 12 May 2022, https://www.hindawi.com/journals/misy/2022/3387598/.  <br>
<a name="Lippens"></a>5.[^](#Lippens): Lippens, S., Martens, J.P, Leman, M., Baets, B., Meyer, H. A COMPARISON OF HUMAN AND AUTOMATIC MUSICAL GENRE CLASSIFICATION. *ResearchGate*. https://www.researchgate.net/publication/200806218_A_Comparison_of_Human_and_Automatic_Musical_Genre_Classification<br> 
